In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import time
from lightgbm import LGBMClassifier
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=d35185f25bea394215fdc3401b53eb18420585f9ff4fab61c05383aff06b7cc7
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


In [ ]:
import math
from math import radians

In [ ]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 5                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [ ]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [ ]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [ ]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [ ]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def scale_to_supervised(train, n_in, dropnan=True):

    # input sequence (t-n, ... t-1)
    if dropnan:
      df = pd.DataFrame(train.iloc[:,:-1])
      df2 = pd.DataFrame(train.iloc[:,-1], index = train.index)
      cols, names = list(), list()
      future = [0]
      n_vars = 1 if type(df) is list else df.shape[1]
      n_vars2 = 1 if type(df2) is list else df2.shape[1]
      for i in range(n_in, 0, -1):
          cols.append(df.shift(i))
          names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
      # forecast sequence (t, t+1, ... t+n)
      for i in future:
          cols.append(df2.shift(-i))
          if i == 0:
              names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
          else:
              names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
      # put it all together
      agg = concat(cols, axis=1)
      agg.columns = names
      # drop rows with NaN values
      agg.dropna(inplace=True)
    else:
      df = pd.DataFrame(train)
      cols, names = list(), list()
      n_vars = 1 if type(df) is list else df.shape[1]
      for i in range(n_in, 0, -1):
          cols.append(df.shift(i))
          names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
      # forecast sequence (t, t+1, ... t+n)
      # put it all together
      agg = concat(cols, axis=1)
      agg.columns = names
      # drop rows with NaN values
      agg.dropna(inplace=True)
      #agg.fillna(0,inplace=True)
    return agg

In [ ]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [ ]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [ ]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [ ]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [ ]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [ ]:
dataset5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52560 entries, 0 to 52559
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DHI       52560 non-null  int64  
 1   DNI       52560 non-null  int64  
 2   WS        52560 non-null  float64
 3   RH        52560 non-null  float64
 4   T         52560 non-null  int64  
 5   TARGET    52560 non-null  float64
 6   SH        52560 non-null  float64
 7   sin_time  52560 non-null  float64
 8   cos_time  52560 non-null  float64
 9   season    52560 non-null  int64  
 10  Time      52560 non-null  float64
dtypes: float64(7), int64(4)
memory usage: 4.4 MB


In [ ]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [ ]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [ ]:
dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
dataset6.head(10)

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,0,0,1.5,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,0,0,1.5,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,0,0,1.6,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,0,0,1.6,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,0,0,1.6,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,0,0,1.5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,0,0,1.5,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,0,0,1.4,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,0,0,1.3,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,0,0,1.3,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것.....



#
# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [ ]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(i) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [ ]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [ ]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [ ]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [ ]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [ ]:
testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
testset6.head(10)

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,0,0,2.8,70.34,-6.1,9.5,0.000000,1.000000,0.0,0.0
1,0,0,2.9,71.97,-6.4,9.5,0.130526,0.991445,0.0,0.0
2,0,0,2.9,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0
3,0,0,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0
4,0,0,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0
5,0,0,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0
6,0,0,2.9,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0
7,0,0,2.9,71.97,-8.7,9.5,0.793353,0.608761,0.0,0.0
8,0,0,2.9,68.56,-9.1,9.5,0.866025,0.500000,0.0,0.0
9,0,0,2.7,70.75,-9.5,9.5,0.923880,0.382683,0.0,0.0


In [ ]:
### 되는 것을 확인했으니 여기에 _to_supervised 까지 해 주면 완성입니다.
### 그리고 완성한 총 코드를 for 구문을 이용해서 0~80까지 불러와주면 끝.

In [ ]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)

    testset = temp.copy()
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]

    # testset7 = RG.series_to_supervised(testset6, n_days, dropnan=False)
    # testset3 = testset2.iloc[-ticks:, :]
    df_test.append(testset6)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,0,0,2.7,34.42,0.0,9.5,0.000000,1.000000,0.0,0.0
1,0,0,2.7,34.17,0.1,9.5,0.130526,0.991445,0.0,0.0
2,0,0,2.7,34.23,0.2,9.5,0.258819,0.965926,0.0,0.0
3,0,0,2.7,33.99,0.3,9.5,0.382683,0.923880,0.0,0.0
4,0,0,2.8,33.97,0.4,9.5,0.500000,0.866025,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
331,0,0,0.8,63.35,13.7,15.5,-0.608761,0.793353,0.0,0.0
332,0,0,0.7,64.82,13.1,15.5,-0.500000,0.866025,0.0,0.0
333,0,0,0.7,66.10,12.8,15.5,-0.382683,0.923880,0.0,0.0
334,0,0,0.6,67.64,12.4,15.5,-0.258819,0.965926,0.0,0.0
